# Simple output analysis of LPJ-GUESS modeling run

Accompanying the Paper Gregor et al. (2025)

The line plots below are interactive, you can zoom in for example.
These plots are just a mere example that you can easily put interactive plots online.

In [1]:
# defaults
resolution = '1800arcsec'
regionname = 'bavaria'
model = 'chelsa-w5e5'

# receives data from snakemake
try:
    from snakemake.script import snakemake

    if snakemake.wildcards:
        print('Reading variables from snakemake variables')
        resolution = snakemake.wildcards['resolution']
        regionname = snakemake.wildcards['regionname']
        model = snakemake.wildcards['model']
    else:
        print('Note: code was run with snakemake. Executing output analysis with defaults')
except ImportError as err:
    print('Note: code was not called from within snakemake')

print('Region:', regionname, ', resolution:', resolution, ', model:', model)
outputdir = f'../../results/lpjguess_outputs_{model}_{resolution}_{regionname}/'
carbon_pools_file = outputdir + 'cpool.out'
gpp_file = outputdir + 'agpp.out'

Note: code was not called from within snakemake
Region: bavaria , resolution: 1800arcsec , model: chelsa-w5e5


In [2]:
import pandas as pd
import output_computations as comp
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe"

# Total carbon content in vegetation (from simulation)

In [3]:
M2_TO_KM2 = 1_000_000
KG_TO_MT = 1_000_000

carbon_pool = pd.read_csv(carbon_pools_file, sep='\\s+').set_index(['Lon', 'Lat', 'Year'])
carbon_pool = comp.get_area_weighted_sum_per_year(carbon_pool, variable_name='VegC') / (M2_TO_KM2 * KG_TO_MT)

fig_cpool = px.line(carbon_pool.reset_index(), x="Year", y="value", title="Vegetation Carbon", labels={"value": "MtC"})
fig_cpool.show()

# Example plots for interactive maps (not from simulation)

In [6]:
cpool_map = pd.read_csv('../../resources/cpool_full_europe.csv', sep='\\s+')
cpool_map = cpool_map[cpool_map['Year'] == 2010]

fig = px.scatter_map(
    cpool_map,
    lat="Lat",
    lon="Lon",
    color="Total",
    hover_name="VegC",
    zoom=3,
    center={"lat": 50, "lon": 14},
    title=r"Total Carbon Content (kgC/m2)",
    color_continuous_scale="viridis",  # Use Viridis colormap
)

fig.update_traces(
    marker=dict(size=15),
    hovertemplate="Lon: %{lon}<br>Lat: %{lat}<br>"
                  "Vegetation Carbon: %{customdata[0]} kgC/m2<br>"
                  "Total Carbon: %{customdata[1]} kgC/m2",
    customdata=cpool_map[["VegC", "Total"]].values
)

fig.show()

# Example plot for interactive map with animation

In [8]:
gpp_map = pd.read_csv('../../resources/gpp_full_europe.csv', sep='\\s+')

fig = px.scatter_map(
    gpp_map,
    lat="Lat",
    lon="Lon",
    animation_frame="Year",
    color="Total",
    title=r"Gross Primary Production (kgC/m2/yr)",
    color_continuous_scale="viridis",
    center={"lat": 57, "lon": 30},
    zoom=1.8,
)

min_value, max_value = 0, 1.5
fig.update_layout(
    coloraxis_colorbar=dict(
        title="GPP",
        tickvals=[min_value, max_value],  # Ensure min/max ticks
        ticktext=[f"{min_value:.2f}", f"{max_value:.2f}"]  # Show formatted min/max
    ),
    coloraxis=dict(
        cmin=min_value,  # Set color scale range min
        cmax=max_value   # Set color scale range max
    )
)

fig.update_traces(
    marker=dict(size=15),
    hovertemplate="Lon: %{lon}<br>Lat: %{lat}<br>"
                  "GPP: %{customdata[0]} kgC/m2/yr",
    customdata=gpp_map[["Total"]].values
)

fig.show()